In [10]:
import pyvista as pv
import numpy as np
import glob
import vtk
import os
import requests

## Auxiliary functions

In [ ]:
ZENODO_BASE_URL = "https://zenodo.org/record/16995252/files/" 
AORTA_VTU_FILES = ["aorta_0.vtu", "aorta_1.vtu", "aorta_2.vtu", "aorta_3.vtu"]

In [13]:
def download_file(file_name, folder_path='.'):
    url = ZENODO_BASE_URL + file_name
    local_path = os.path.join(folder_path, file_name)
    
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        print(f"ERROR: could not download {file_name} (status {r.status_code})")
        return False
    
    with open(local_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"{file_name} downloaded successfully.")
    return True

def create_VTKlegacy_files(folder_path, list_of_files, extension='.vtu', expected_files=None):
    """
    For each file in AORTA_VTU_FILES, check if it exists in folder_path (e.g., 'VTK/').
    If not, download it from Zenodo. Then convert to VTK legacy format and write a list of output files and their time values.
    """

    vtu_files = []
    for fname in AORTA_VTU_FILES:
        file_path = os.path.join(folder_path, fname)
        if not os.path.exists(file_path):
            print(f"{file_path} not found locally. Attempting to download from Zenodo...")
            success = download_file(fname, folder_path)
            if not success:
                print(f"Skipping {file_path} (could not download).")
                continue
            if not os.path.exists(file_path):
                print(f"Downloaded file {file_path} still not found. Skipping.")
                continue
        vtu_files.append(file_path)

    if not vtu_files:
        print(f"No VTU files found in {folder_path} and could not download any from Zenodo.")
        return

    with open(list_of_files, 'w') as file:
        for file_ in vtu_files:
            try:
                mesh = pv.read(file_)
            except Exception as e:
                print(f"Failed to read {file_}: {e}")
                continue

            base_name = os.path.splitext(os.path.basename(file_))[0]
            file_name = base_name + '.vtk'

            # Save the VTK legacy file
            writer = vtk.vtkDataSetWriter()
            writer.SetFileTypeToASCII()
            writer.SetInputData(mesh)
            writer.SetFileName(file_name)
            writer.SetFileVersion(42)
            writer.SetWriteArrayMetaData(False)
            writer.Write()
            print(mesh['TimeValue'][0], file_name)

            file.write(f'{file_name}\t{mesh["TimeValue"][0]}\n')

def sort_txt_file(file_path):
  # Read the contents of the file
  with open(file_path, 'r') as file:
    lines = file.readlines()

  # Sort the lines based on the second column
  sorted_lines = sorted(lines, key=lambda line: float(line.split()[1]))

  # Write the sorted lines back to the file
  with open(file_path, 'w') as file:
    file.writelines(sorted_lines)

def create_seed_file(points, seedfile):
  with open(seedfile, 'w') as file:
    for i, point in enumerate(points):
      file.write(f'{point[0]} {point[1]} {point[2]}\n')
      
def sample_points_inside_mesh(mesh, n_points, threshold=1e-3):
    xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds
    points_inside = []
    # Keep sampling until you have enough points inside
    while len(points_inside) < n_points:
        n_sample = max(n_points - len(points_inside), n_points // 2)
        points = np.column_stack([
            np.random.uniform(xmin, xmax, n_sample),
            np.random.uniform(ymin, ymax, n_sample),
            np.random.uniform(zmin, zmax, n_sample)
        ])
        enclosed = pv.PolyData(points).select_enclosed_points(mesh, tolerance=0.0)
        mask = enclosed['SelectedPoints'] == 1
        filtered_points = enclosed.points[mask]
        distances = pv.PolyData(filtered_points).compute_implicit_distance(mesh)
        mask = np.abs(distances['implicit_distance']) > threshold
        selected_points = filtered_points[mask]
        points_inside.extend(selected_points.tolist())
        print(f'Sampled {len(points_inside)} / {n_points} points inside the mesh...')
    return np.array(points_inside[:n_points]), distances

## Prepare geometry to work with VTK-m

In [14]:
# The function create_VTKlegacy_files converts all VTU files in a given folder to VTK legacy format.
# It searches for files with the '.vtu' extension in the specified folder and its subfolders.
# For each VTU file found, it reads the mesh, writes it as a VTK legacy file (ASCII format), and saves a list of files in txt format.
# The output file is named 'flowmap_<suffix>.vtk', where <suffix> is derived from the original filename.
# It also records the mapping between the new VTK filename and the mesh's 'TimeValue' attribute in a text file (list_of_files.txt).
# The function returns 0 after processing all files.
create_VTKlegacy_files('./VTK', 'list_of_files.txt', extension='.vtu')

./VTK/aorta_3.vtu not found locally. Attempting to download from Zenodo...
aorta_3.vtu downloaded successfully.
9.01 aorta_0.vtk
9.02 aorta_1.vtk
9.03 aorta_2.vtk
9.04 aorta_3.vtk


### Sort list of files

In [9]:
# The function sort_txt_file reads a text file, sorts its lines based on the second column (converted to float), 
# and writes the sorted lines back to the file. This is useful for organizing files by a numerical value, 
# such as time or other scalar attributes.
sort_txt_file('./list_of_files.txt')

It is possible to work with both .vtk and image files (.vts). To convert vtk to image files,  

In [6]:
# Define the folder path
folder_path = "./"
destination_path = './'

# Get a list of all VTK files in the folder
_files = [file for file in os.listdir(folder_path) if file.endswith(".vtk")]

# Loop over each VTK file
for i, _file in enumerate(_files):
  # Load the file
  mesh = pv.read(os.path.join(folder_path, _file))

  Coords_0  = mesh.points
  scaling=1
  image_box = [scaling*(np.max(Coords_0[:,0])-np.min(Coords_0[:,0])),\
          scaling*(np.max(Coords_0[:,1])-np.min(Coords_0[:,1])),\
          scaling*(np.max(Coords_0[:,2])-np.min(Coords_0[:,2]))]

  offset = 0
  orig_image =[np.min(Coords_0,axis=0)[0]+offset,\
          np.min(Coords_0,axis=0)[1]+offset,\
          np.min(Coords_0,axis=0)[2]+offset]

  edge_size = .5e-3
  image_sizex=edge_size
  image_sizey=edge_size
  image_sizez=edge_size

  nx = int(image_box[0]/image_sizex)
  ny = int(image_box[1]/image_sizey)
  nz = int(image_box[2]/image_sizez)

  res_x = image_box[0]/nx
  res_y = image_box[1]/ny
  res_z = image_box[2]/nz

  dims=(nx, ny, nz)
  spa=(res_x,res_y,res_z)
  orig=(orig_image[0],orig_image[1],orig_image[2])
  image = pv.ImageData(dimensions=dims,spacing=spa,origin=orig)

  mesh=mesh.cell_data_to_point_data()
  probed_image=image.sample(mesh, mark_blank=False)
  array_data=probed_image.point_data
  #probed_image.clear_arrays()
  probed_image['U']=array_data['U']
  probed_image=probed_image.point_data_to_cell_data()
  mesh = probed_image

  # Save the VTK legacy file
  writer = vtk.vtkDataSetWriter()
  writer.SetFileTypeToASCII()
  writer.SetInputData(mesh)
  # Set the output file name with .vts extension
  output_file = _file.rsplit('.vtk')[0]+'.vts'
  writer.SetFileName(folder_path + '/' + output_file)
  print(i, _file,  output_file)
  writer.SetFileVersion(42)
  writer.SetWriteArrayMetaData(0)
  writer.Write()

0 aorta_1.vtk aorta_1.vts
1 aorta_2.vtk aorta_2.vts
2 aorta_3.vtk aorta_3.vts
3 aorta_0.vtk aorta_0.vts


Now it is possible to run the computation using this image files instead of VTK files (faster, but probably less accurate). To do so, it is neccesary to change the extension of the files in 'list_of_files.txt'

In [21]:
with open('list_of_files.txt', 'r') as file:
    lines = file.readlines()

with open('list_of_vts_files.txt', 'w') as file:
    for line in lines:
        file.write(line.replace('.vtk', '.vts'))

and change accordingly the advection_settings.json file

## Generate random internal seeds

In [7]:
fov_mesh = pv.read('./fov.vtu')
surface = fov_mesh.extract_surface()

In [8]:
surface

PolyData (0x7fb40692a560)
  N Cells:    177586
  N Points:   88795
  N Strips:   0
  X Bounds:   -8.381e-02, -4.283e-02
  Y Bounds:   -2.869e-02, 6.421e-02
  Z Bounds:   -7.500e-02, -5.000e-03
  N Arrays:   11

In [9]:
# Create random number of seeds within the peeled surface
num_seeds = 10000

points, _ = sample_points_inside_mesh(surface, num_seeds, threshold=0.5e-3)


Sampled 2226 / 10000 points inside the mesh...
Sampled 3917 / 10000 points inside the mesh...
Sampled 5255 / 10000 points inside the mesh...
Sampled 6314 / 10000 points inside the mesh...
Sampled 7451 / 10000 points inside the mesh...
Sampled 8489 / 10000 points inside the mesh...
Sampled 9594 / 10000 points inside the mesh...
Sampled 10695 / 10000 points inside the mesh...


In [10]:
pv.PolyData(points).save('internal_seeds.vtp')

In [11]:
create_seed_file(points, 'internal_seeds.txt')

## Generate seeds for reseeding at entrance

In [12]:
inflow_mesh = pv.read('./inflow.vtu')
inflow_surface = inflow_mesh.extract_surface()

In [13]:
num_seeds = 1000

points, _ = sample_points_inside_mesh(inflow_surface, num_seeds, threshold=0.005e-3)


Sampled 785 / 1000 points inside the mesh...
Sampled 1179 / 1000 points inside the mesh...


In [14]:
pv.PolyData(points).save('inflow_seeds.vtp')

In [15]:
create_seed_file(points, 'entrance_seeds.txt')